# Accounting for optional clients

TODO

### Prize-collecting VRP

We now have a basic familiarity with PyVRP's `Model` interface, but have not seen some of its additional features yet.
In this short section we will discuss _optional_ clients, which offer a reward (a prize) when they are visited, but are not required for feasibility.
This VRP variant is often called a prize-collecting VRP, and PyVRP supports this out-of-the-box.

Let's stick to the multiple depot setting, and also define a `PRIZES` list that provides the prizes of visiting each client.

In [ ]:
# fmt: off
PRIZES = [
    0,    # location 0 - a depot
    0,    # location 1 - a depot
    334,  # location 2
    413,  # location 3
    295,  # location 4
    471,  # location 5
    399,  # location 6
    484,  # location 7
    369,  # location 8
    410,  # location 9
    471,  # location 10
    382,  # location 11
    347,  # location 12
    380,  # location 13
    409,  # location 14
    302,  # location 15
    411,  # location 16
]
# fmt: on

When modelling optional clients, it is important to provide both a reward (the `prize` argument to `add_client`), and to mark the client as optional by passing `required=False`:

In [ ]:
m = Model()

for idx in range(2):
    depot = m.add_depot(x=COORDS[idx][0], y=COORDS[idx][1])
    m.add_vehicle_type(
        2,
        start_depot=depot,
        end_depot=depot,
        tw_early=TIME_WINDOWS[idx][0],
        tw_late=TIME_WINDOWS[idx][1],
    )


for idx in range(2, len(COORDS)):
    m.add_client(
        x=COORDS[idx][0],
        y=COORDS[idx][1],
        tw_early=TIME_WINDOWS[idx][0],
        tw_late=TIME_WINDOWS[idx][1],
        prize=PRIZES[idx],
        required=False,
    )

for frm_idx, frm in enumerate(m.locations):
    for to_idx, to in enumerate(m.locations):
        distance = abs(frm.x - to.x) + abs(frm.y - to.y)  # Manhattan
        duration = DURATION_MATRIX[frm_idx][to_idx]
        m.add_edge(frm, to, distance=distance, duration=duration)

In [ ]:
res = m.solve(stop=MaxRuntime(1), display=False)  # one second
print(res)

In [ ]:
_, ax = plt.subplots(figsize=(8, 8))
plot_solution(res.best, m.data(), plot_clients=True, ax=ax)

Some clients are not visited in the figure above.
These clients are too far from other locations for their prizes to be worth the additional travel cost of visiting.
Thus, PyVRP's solver opts not to visit such optional clients.